In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
import datetime
import os

In [3]:
assembled_list = os.listdir('./assembled-data/')

In [75]:
for file in assembled_list: 
    
    df = pd.read_csv(f'./assembled-data/{file}', names=['cik', 'fn_name', 'end', 'val', 'filed'])

    df = df.drop_duplicates()

    df.end = pd.to_datetime(df.end)
    df.filed = pd.to_datetime(df.filed)

    cik_list = df.cik.unique()

    # creates a seperate csv file for each company that includes all years of data
    for cik in cik_list: 

        df1 = df[df.cik == cik]

        df_set = pd.DataFrame()
        years = pd.Series(df1.end).dt.year.unique()
        
        for year in years:

            df2 = df1[df1.end.dt.strftime('%Y') == str(year)]
            df3 = df2.sort_values(by=['fn_name', 'filed'])
            df4 = df3.drop_duplicates(subset='fn_name', keep='last')
            df5 = df4[['fn_name', 'val']]
            df6 = df5.set_index('fn_name')
            df7 = df6.transpose()

            df7['reporting_for'] = year
            cols = df7.columns.to_list()
            cols = cols[-1:] + cols[:-1]
            df7 = df7[cols]

            df7['cik'] = cik
            cols = df7.columns.to_list()
            cols = cols[-1:] + cols[:-1]
            df7 = df7[cols]

            df_set = pd.concat([df_set, df7], ignore_index=True)

        df_set.to_csv(f'./company-fin-sets/{cik}.csv', mode='w', index=False, header=True)